In [1]:
%load_ext autoreload
%autoreload 2

# Villin subdomain HP-35

Chicken villin subdomain headpiece 35 (pdbid:1vii)

In [2]:
import molsysmt as msm

In [3]:
molecular_system = msm.convert('pdbid:1vii', to_form='molsysmt.MolSys')
molecular_system = msm.build.remove_hydrogens(molecular_system)
molecular_system = msm.build.add_terminal_cappings(molecular_system, N_terminal='ACE', C_terminal='NME')
molecular_system = msm.build.add_hydrogens(molecular_system, pH=7.4)

In [4]:
msm.info(molecular_system)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_peptides,n_frames
molsysmt.MolSys,605,38,1,1,1,1,1,1


In [5]:
msm.view(molecular_system, standardize=True)

NGLWidget()

In [6]:
msm.convert(molecular_system, to_form='villin_hp35_vacuum.msmpk')

'villin_hp35_vacuum.msmpk'

In [7]:
molecular_system = msm.build.solvate([molecular_system, {'forcefield':'AMBER14', 'water_model':'TIP3P'}],
                          box_geometry='truncated octahedral', clearance='14.0 angstroms',
                          to_form='molsysmt.MolSys', engine="OpenMM", verbose=False)

In [8]:
msm.info(molecular_system)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_peptides,n_frames
molsysmt.MolSys,5116,1543,1506,3,1506,3,1503,2,1,1


In [9]:
msm.convert(molecular_system, to_form='villin_hp35_solvated.msmpk')

'villin_hp35_solvated.msmpk'

In [21]:
import openmm as mm
from openmm import app
from openmm import unit
from sys import stdout
from mdtraj.reporters import HDF5Reporter

modeller = msm.convert(molecular_system, to_form='openmm.Modeller')
forcefield = app.ForceField("amber14-all.xml", "amber14/tip3p.xml")
system = forcefield.createSystem(topology, nonbondedMethod=app.PME, nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)
platform = mm.Platform.getPlatformByName('CUDA')
simulation = app.Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
simulation.reporters.append(app.StateDataReporter(stdout, 50000, step=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(app.DCDReporter('villin_hp35_solvated.dcd', 50000, enforcePeriodicBox=True))
simulation.reporters.append(HDF5Reporter('villin_hp35_solvated.h5', 50000))
simulation.step(1000000)
simulation.reporters[2].close()
final_positions = simulation.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(modeller.topology, final_positions, open('villin_hp35_solvated_last.pdb', 'w'))

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
50000,-62709.05659608188,298.930331423212
100000,-62883.25972108188,296.31811782582236
150000,-63003.57222108188,308.1702234403577
200000,-62976.43159608188,293.8885228940937
250000,-63062.74409608188,299.0748258229993
300000,-63174.55659608188,297.2483345256471
350000,-62728.66597108188,303.86319552401187
400000,-62363.32222108188,303.9745092850282
450000,-63090.88472108188,295.60701750184325
500000,-62835.04097108188,306.01264158375136
550000,-63009.29097108188,295.5271110521145
600000,-62920.13472108188,299.80539813376305
650000,-62764.46284608188,296.37558281773886
700000,-62544.85347108188,298.86950362911807
750000,-63011.05659608188,292.5764601256301
800000,-62565.41597108188,307.78915748341524
850000,-62496.27534608188,301.0442065082505
900000,-62841.05659608188,302.8333918048054
950000,-62970.94722108188,304.27691481181296
1000000,-62645.69722108188,304.6894773866675


## References

[Piana, Stefano, Kresten Lindorff-Larsen, and David E. Shaw. “Protein Folding Kinetics and Thermodynamics from Atomistic Simulation.” Proceedings of the National Academy of Sciences 109, no. 44 (2012): 17845–50.](https://doi.org/10.1073/pnas.1201811109)

[Freddolino, Peter L., Christopher B. Harrison, Yanxin Liu, and Klaus Schulten. “Challenges in Protein-Folding Simulations.” Nature Physics 6, no. 10 (2010): 751–58.](https://doi.org/10.1038/nphys1713)

[Ensign, Daniel L., Peter M. Kasson, and Vijay S. Pande. “Heterogeneity Even at the Speed Limit of Folding: Large-Scale Molecular Dynamics Study of a Fast-Folding Variant of the Villin Headpiece.” Journal of Molecular Biology 374, no. 3 (2007): 806–16.](https://doi.org/10.1016/j.jmb.2007.09.069)
    
[Mori, Toshifumi, and Shinji Saito. “Molecular Mechanism Behind the Fast Folding/Unfolding Transitions of Villin Headpiece Subdomain: Hierarchy and Heterogeneity.” The Journal of Physical Chemistry B 120, no. 45 (2016): 11683–91.](https://doi.org/10.1021/acs.jpcb.6b08066)

[Bunagan, Michelle R., Jianmin Gao, Jeffery W. Kelly, and Feng Gai. “Probing the Folding Transition State Structure of the Villin Headpiece Subdomain via Side Chain and Backbone Mutagenesis.” Journal of the American Chemical Society 131, no. 21 (2009): 7470–76.](https://doi.org/10.1021/ja901860f)

[Meng, Wenli, Bing Shan, Yuefeng Tang, and Daniel P. Raleigh. “Native like Structure in the Unfolded State of the Villin Headpiece Helical Subdomain, an Ultrafast Folding Protein.” Protein Science 18, no. 8 (2009): 1692–1701.](https://doi.org/10.1002/pro.152)

[Wang, Ercheng, Peng Tao, Jun Wang, and Yi Xiao. “A Novel Folding Pathway of the Villin Headpiece Subdomain HP35.” Physical Chemistry Chemical Physics 21, no. 33 (2019): 18219–26.](https://doi.org/10.1039/c9cp01703h)

[Huang, Jing, Sarah Rauscher, Grzegorz Nawrocki, Ting Ran, Michael Feig, Bert L de Groot, Helmut Grubmüller, and Alexander D MacKerell. “CHARMM36m: An Improved Force Field for Folded and Intrinsically Disordered Proteins.” Nature Methods 14, no. 1 (2017): 71–73.](https://doi.org/10.1038/nmeth.4067)

[Eastman, Peter, Mark S. Friedrichs, John D. Chodera, Randall J. Radmer, Christopher M. Bruns, Joy P. Ku, Kyle A. Beauchamp, et al. “OpenMM 4: A Reusable, Extensible, Hardware Independent Library for High Performance Molecular Simulation.” Journal of Chemical Theory and Computation 9, no. 1 (2013): 461–69.](https://doi.org/10.1021/ct300857j)

[Harvey, M J, G Giupponi, and G De Fabritiis. “ACEMD: Accelerating Biomolecular Dynamics in the Microsecond Time Scale.” Journal of Chemical Theory and Computation 5, no. 6 (2009): 1632–39.](https://doi.org/10.1021/ct9000685)